In [2]:
# Basic packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import boto3

import os

In [2]:
!pip install joblib

In [3]:
#!pip install sagemaker==1.72.0

# Sagemaker
import sagemaker

In [4]:
X_train = pd.read_csv('../output/xtrain.csv', header = None)
X_test =  pd.read_csv('../output/xtest.csv', header = None)

y_train = pd.read_csv('../output/ytrain.csv', header = None)
y_test =  pd.read_csv('../output/ytest.csv', header = None)

In [5]:
X_test.head()

,0,1,2,3,4,5,6,7,8,9,...,490,491,492,493,494,495,496,497,498,499
0,44,1171,23,763,44,1415,454,1,23,2,...,0,0,0,0,0,0,0,0,0,0
1,166,166,13,166,166,22,433,414,5,1,...,0,0,0,0,0,0,0,0,0,0
2,21,42,33,16,12,120,1422,1578,401,195,...,0,0,0,0,0,0,0,0,0,0
3,1569,600,1,663,1562,142,320,172,28,6,...,0,0,0,0,0,0,0,0,0,0
4,11,238,44,330,633,1680,978,61,1,316,...,0,0,0,0,0,0,0,0,0,0


## 2. Upload the data to S3

In [6]:
len(y_train)
#len(X_train)

800

In [7]:
data_dir = '../output/'

In [8]:
aws_data = pd.concat([y_train, X_train], axis=1)

aws_data.to_csv(os.path.join(data_dir, 'train.csv'), header=False, index=False)

In [9]:
aws_data.head()

,0,0,1,2,3,4,5,6,7,8,...,490,491,492,493,494,495,496,497,498,499
0,0,7,93,819,1,269,201,1856,180,88,...,0,0,0,0,0,0,0,0,0,0
1,0,1368,122,1275,2,2577,758,186,391,93,...,0,0,0,0,0,0,0,0,0,0
2,0,994,894,637,849,2152,69,51,40,821,...,0,0,0,0,0,0,0,0,0,0
3,0,7,11,51,99,1,48,26,4,3,...,0,0,0,0,0,0,0,0,0,0
4,1,1512,1049,316,27,4,178,308,688,11,...,0,0,0,0,0,0,0,0,0,0


### 2.2. Uploading training data

In [10]:
sagemaker_session = sagemaker.Session()
bucket = sagemaker_session.default_bucket()
role = sagemaker.get_execution_role()

In [11]:
data_dir = "../output"
prefix = 'positiveness-lyrics'

# upload all data to S3
data = sagemaker_session.upload_data(path=data_dir, bucket=bucket, key_prefix=prefix)
print(data)

s3://sagemaker-us-east-2-890904620905/positiveness-lyrics


In [12]:
# confirm that data is in S3 bucket
empty_check = []
for obj in boto3.resource('s3').Bucket(bucket).objects.all():
    empty_check.append(obj.key)
    print(obj.key)

assert len(empty_check) !=0, 'S3 bucket is empty.'
print('Test passed!')

positiveness-lyrics/sagemaker-scikit-learn-2021-04-29-19-12-27-071/profiler-output/system/incremental/2021042919/1619723640.algo-1.json
positiveness-lyrics/sagemaker-scikit-learn-2021-04-29-19-12-27-071/profiler-output/system/incremental/2021042919/1619723700.algo-1.json
positiveness-lyrics/sagemaker-scikit-learn-2021-04-29-19-42-46-147/profiler-output/system/incremental/2021042919/1619725440.algo-1.json
positiveness-lyrics/sagemaker-scikit-learn-2021-04-29-19-42-46-147/profiler-output/system/incremental/2021042919/1619725500.algo-1.json
positiveness-lyrics/sagemaker-scikit-learn-2021-04-29-19-52-24-687/profiler-output/system/incremental/2021042919/1619726040.algo-1.json
positiveness-lyrics/sagemaker-scikit-learn-2021-04-29-19-52-24-687/profiler-output/system/incremental/2021042919/1619726100.algo-1.json
positiveness-lyrics/sagemaker-scikit-learn-2021-04-29-20-13-45-386/debug-output/training_job_end.ts
positiveness-lyrics/sagemaker-scikit-learn-2021-04-29-20-13-45-386/output/model.tar.

### Train Base Model - naiveBayes

In [30]:
# your import and estimator code, here
from sagemaker.sklearn.estimator import SKLearn

output_path = 's3://{}/{}'.format(bucket, prefix)

# instantiate our custom SKLearn estimator
estimator = SKLearn(entry_point='train.py',
                    source_dir='../utilis/base_model',
                    role=role,
                    train_instance_count=1,
                    train_instance_type='ml.c4.xlarge',
                    output_path=output_path,
                    py_version='py3',
                    framework_version='0.23-1',
                    sagemaker_session=sagemaker_session
                   
                   )

In [31]:
%%time

# Train your estimator on S3 training data
estimator.fit({'train': data})

's3_input' class will be renamed to 'TrainingInput' in SageMaker Python SDK v2.


2021-04-29 20:13:45 Starting - Starting the training job...
2021-04-29 20:13:47 Starting - Launching requested ML instances......
2021-04-29 20:15:00 Starting - Preparing the instances for training.........
2021-04-29 20:16:19 Downloading - Downloading input data...
2021-04-29 20:17:01 Training - Downloading the training image..2021-04-29 20:17:23,636 sagemaker-containers INFO     Imported framework sagemaker_sklearn_container.training
2021-04-29 20:17:23,639 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2021-04-29 20:17:23,650 sagemaker_sklearn_container.training INFO     Invoking user training script.
2021-04-29 20:17:24,072 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2021-04-29 20:17:27,099 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2021-04-29 20:17:27,112 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2021-04-29 20:17:27,122 sagemak

### Deploy and evaluate base model

In [32]:
predictor = estimator.deploy(initial_instance_count=1, instance_type='ml.t2.medium')

Parameter image will be renamed to image_uri in SageMaker Python SDK v2.


-------------------!

In [44]:
from sklearn.metrics import accuracy_score

# generate
y_preds  = predictor.predict(X_test)
print(y_preds)
accuracy = accuracy_score(y_preds, y_test)

print(f"The estimator had an accuracy of {accuracy:.2%} in the test set")

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 1 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0
 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
The estimator had an accuracy of 51.50% in the test set


## Main model - LSTM Network

In [49]:
!pip install torch

In [14]:
import torch
import torch.utils.data

# Turn the input pandas dataframe into tensors
train_y = torch.from_numpy(aws_data[[0]].values).float().squeeze()
train_X = torch.from_numpy(aws_data.drop([0], axis=1).values).long()

# Build the dataset
train_ds = torch.utils.data.TensorDataset(train_X, train_y)
# Build the dataloader
train_dl = torch.utils.data.DataLoader(train_ds, batch_size=50)

## Clean up Resources


In [ ]:
predictor.endpoint

In [ ]:
boto3.client('sagemaker').delete_endpoint(EndpointName=predictor.endpoint)


In [ ]:
bucket_to_delete = boto3.resource('s3').Bucket(bucket)
bucket_to_delete.objects.all().delete()